# Calculate similarity between category based on 1 vs all classifier


## 1. Set up

In [ ]:
import os
import json
import glob
import scipy
import numpy as np

# import preprocess func from processor module
from processor import preprocess
from processor import DataSet

from keras.models import model_from_json
from keras.models import model_from_yaml
from keras.utils import np_utils
from keras.preprocessing import image

import pandas as pd
import glob

In [ ]:
# set root as ../
import sys
sys.path.append("../")

In [ ]:
BASE_MODEL_DIR = "./trained_model/inceptionv3/"
BASE_NAME = "model178_1vsAll"
CODE_EXTRACTOR=""
BASE_CODE_NAME = "code_" + BASE_NAME

In [ ]:
DATA_DIR = ""

BASE_DATA_DIR= os.path.normpath(os.path.join(DATA_DIR, ''))

CATSTORE_PATH = ""

In [ ]:
VALID_DATA_DIR = os.path.normpath(os.path.join(BASE_DATA_DIR, "valid"))

In [ ]:
VALID_DISTANCE_DICT_PATH=""


VALID_DF_PATH=""

### Get model from S3

In [ ]:
REGION_NAME = "ap-northeast-1"
DEFAULT_OUTPUT = "text"

DIST_DATA_DIR = os.path.normpath(BASE_DATA_DIR)

os.environ['AWS_DEFAULT_REGION'] = REGION_NAME
os.environ['AWS_DEFAULT_OUTPUT'] = DEFAULT_OUTPUT

!aws s3 sync 

## Calc similarity and store df

In [ ]:
from one_vs_all import ModelBinder
from codeextractor import DNNCodeExtractor

def ModelBinder_create(base_model_name = BASE_CODE_NAME, basedir = BASE_MODEL_DIR, extractor_path=CODE_EXTRACTOR):
    return ModelBinder(base_model_name, basedir, DNNCodeExtractor.create_from(extractor_path))

### For 178 category

In [ ]:
from modelutils import dir2filedict
fdict = dir2filedict(VALID_DATA_DIR)
catkeys = sorted(list(fdict.keys()))

### For virtual category case (138 or 159)

In [ ]:
from category import VirtualCategories
vc = VirtualCategories.from_file(CATSTORE_PATH)
catkeys = vc.keys()
fdict = vc.list_dict(VALID_DATA_DIR)

## Predict score and store to df

In [ ]:
ftuppls = [(key, file) for key in fdict.keys() for file in fdict[key]]

In [ ]:
labels = [tup[0] for tup in ftuppls]

In [ ]:
paths = [tup[1] for tup in ftuppls]

In [ ]:
len(labels), len(paths)

In [ ]:
ens = ModelBinder_create()

In [ ]:
ens.load_all_models(catkeys)

In [ ]:
%%time
df = ens.predict(paths)

In [ ]:
df['label'] = labels

In [ ]:
df.to_pickle(VALID_DF_PATH)

## Calculate similarity from score df

In [ ]:
class SimilarityCalculator:
    def __init__(self, df, thrshold=0.5):
        self.df = df
        self.rmiss = {}
        self.thrshold = thrshold
    def combinekey(self, fromkey, tokey):
        return "{0}~{1}".format(fromkey, tokey)
    def saveAll(self):
        for key in self.df.columns[:-2]:
            self.saveNaxx(key)
    def saveNaxx(self, fromkey):
        targetdf = self.df[self.df['label'] == fromkey]
        
        Nab = (targetdf[targetdf.columns[:-2]] > self.thrshold).sum()
        Na = len(targetdf)
        
        Nab_a = Nab/Na
        
        otherkeys = Nab_a.index
        list(map(lambda otherkey: self.add(fromkey, otherkey, Nab_a[otherkey]), otherkeys))

        
    def add(self, fromkey, tokey, val):
        key = self.combinekey(fromkey, tokey)
        self.rmiss.setdefault(key, 0)
        self.rmiss[key] += val
    def distance(self, fromkey, tokey):
        key1 = self.combinekey(fromkey, tokey)
        key2 = self.combinekey(tokey, fromkey)
        return self.rmiss[key1] + self.rmiss[key2]
        


In [ ]:
sim = SimilarityCalculator(df)

In [ ]:
sim.saveAll()

In [ ]:
keys = df.columns[:-2]

In [ ]:
import itertools

In [ ]:
len(list(itertools.combinations(keys, 2)))

In [ ]:
len(list(itertools.combinations(keys, 2)))

In [ ]:
distsdict = {key1: [sim.distance(key1, key2) for key2 in keys] for key1 in keys}

In [ ]:
distdf = pd.DataFrame(distsdict)

In [ ]:
distdf.index = keys

In [ ]:
distdf.to_pickle(VALID_DISTANCE_DICT_PATH)

## Send result to S3

In [ ]:


DIST_FILE=''
DIST_DIR=''


In [ ]:
DIST_PATH=os.path.join(DIST_DIR, DIST_FILE)

In [ ]:
os.link(VALID_DISTANCE_DICT_PATH, DIST_PATH)

In [ ]:
S3PATH = "" + DIST_PATH

In [ ]:
!aws s3 cp  {DIST_PATH} {S3PATH} 

# Misc evaluation

In [ ]:
from visualize import plot_image_list
def plot_cat(df, cat, thrld, origin=0):
    filtered = df[df[cat] > thrld].sort_values(by=cat, ascending=False)
    for i in range(0, 50, 5):
        print(",".join([os.path.basename(os.path.dirname(v)) for v in filtered['filepaths'].values[i+origin:(i+5+origin)]]))
    for i in range(0, 50, 5):
        print(",".join(["{0:.4f}".format(v) for v in filtered[cat].values[origin+i:(origin+5+i)]]))
    plot_image_list(filtered["filepaths"].values[origin:(origin+50)])

In [ ]:
#plot_cat(df, '41', 0.1)

### load df from file

In [ ]:
import os
import json
import glob
import scipy
import numpy as np

import pandas as pd
import glob

In [ ]:
distdf = pd.read_pickle(VALID_DISTANCE_DICT_PATH)

In [ ]:
CATEGORY_DICT_CSV=os.path.join(BASE_DATA_DIR, "")

In [ ]:
from category import load_category_dict
catdict = load_category_dict(CATEGORY_DICT_CSV)

In [ ]:
def listdist(targetkey):
    dist = distdf[targetkey].sort_values(ascending=False)
    print(dist.index[0:20])
    # dist.index = [catdict[int(idx)] for idx in dist.index]
    dist.index = [vc.name(idx) for idx in dist.index]
    return dist[0:20]

In [ ]:
def listdist15(targetkey):
    dist = distdf[targetkey].sort_values(ascending=False)
    dist = dist[dist > 1.5]
    
    for idx in dist.index:
        print("{0}:   {2:.4f} {1}".format(idx, vc.name(idx), dist[idx]))


In [ ]:
listdist15('76')

In [ ]:
listdist('110')

In [ ]:
name2id = {catdict[key]:key for key in catdict.keys()}

In [ ]:
name2id['']

In [ ]:
vc.name("118_155")

In [ ]:
listdist('118_155')

In [ ]:
def catlist2names(catlist, catdictst):
    return ["+".join(cats2) for cats2 in 
               [map(lambda cat: catdictst[cat], cats) for cats in catlist]]

In [ ]:
from category import merge_categories_above

In [ ]:
catdictst = {key:vc.name(key) for key in vc.keys()}

In [ ]:
catlist = merge_categories_above(distdf, 1.89)

In [ ]:
len(catlist)

In [ ]:
catlist2names(catlist, catdictst)

In [ ]:
catlist19 = merge_categories_above(distdf, 1.9)

In [ ]:
len(catlist19)

In [ ]:
catlist2names(catlist19, catdictst)

In [ ]:
HIGH_CATEGORY_PATH="trained_model/high_cats19.dat"

In [ ]:
with open(HIGH_CATEGORY_PATH, "wb") as f:
    pickle.dump(catlist19, f)

In [ ]:
catlist19[0:20]

### Choose lonely subcategory

In [ ]:
catlist = merge_categories_above(distdf, 1.8)

In [ ]:
len(catlist)

In [ ]:
catlist2names(catlist, catdictst)

In [ ]:
singlelist = [cat for cat in catlist if len(cat) == 1]

In [ ]:
len(singlelist)

In [ ]:
distdf['176']['176']

In [ ]:
[(cat[0], catdictst[cat[0]]) for cat in singlelist]

In [ ]:
def listdist(targetkey):
    dist = distdf[targetkey].sort_values(ascending=False)
    dist.index = [vc.name(idx) for idx in dist.index]
    return dist[0:20]

In [ ]:
listdist("10")

In [ ]:
listdist("101")

In [ ]:
listdist("102")

In [ ]:
listdist("104")

In [ ]:
listdist("135")

In [ ]:
listdist("176")

In [ ]:
listdist("0_128")

In [ ]:
listdist("107")

In [ ]:
listdist("171")

In [ ]:
listdist("93")

In [ ]:
listdist("33")

In [ ]:
listdist("108_72")

In [ ]:
def listdistgt(targetkey, val):
    dist = distdf[targetkey][distdf[targetkey] >= val].sort_values(ascending=False)
    dist.index = [vc.name(idx) for idx in dist.index]
    return dist


In [ ]:
listdistgt('33', 1.5)

In [ ]:
listdistgt('93', 1.2)

In [ ]:
distdf['93'][distdf['93'] >= 1.5].index

In [ ]:
listdistgt("108_72", 1.5)

In [ ]:
distdf['108_72'][distdf['108_72'] >= 1.5].index

In [ ]:
TRAIN_DATA_DIR_FOR_1VSALL= os.path.normpath(os.path.join(DATA_DIR, ''))


In [ ]:
def count_train_files(targetcat):
    return len(list(glob.glob(os.path.join(os.path.join(TRAIN_DATA_DIR_FOR_1VSALL, "train"), targetcat)+"/*.jpg")))

In [ ]:
count_train_files("135")

In [ ]:
# not handle virtual category, so the category with under score just return 0
[(cat[0], catdictst[cat[0]], distdf[cat[0]][cat[0]], count_train_files(cat[0])) for cat in singlelist]

In [ ]:
tups = [(cat[0], catdictst[cat[0]], distdf[cat[0]][cat[0]], count_train_files(cat[0])) for cat in singlelist]

In [ ]:
from operator import itemgetter

In [ ]:
sorted(tups, key=itemgetter(2))

In [ ]:
os.path.join(BASE_DATA_DIR, "train")

# Try and error

In [ ]:
sim.distance(targetkey, tokey)

In [ ]:
dists = {}

In [ ]:
dists = {"{0}~{1}".format(*pair): sim.distance(pair[0], pair[1])+sim.distance(pair[1], pair[0]) for pair in itertools.combinations(keys, 2)}

In [ ]:
list(dists.keys())[0:3]

In [ ]:
#with open(VALID_DISTANCE_DICT_PATH, mode='wb') as f:
#    pickle.dump(dists, f)

In [ ]:
dists['109~93']

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
plt.hist(list(dists.values()), bins=10)

In [ ]:
tmp = pd.DataFrame({"a":[1, 2, 3], "b":[4, 5, 6]})
tmp

In [ ]:
tmp.index = ["d", "e", "f"]

In [ ]:
tmp

In [ ]:
dists = {"{0}".format(*pair): sim.distance(pair[0], pair[1])+sim.distance(pair[1], pair[0]) for pair in itertools.combinations(keys, 2)}

In [ ]:
dist = distdf["93"].sort_values(ascending=False)

In [ ]:
dist.index = [vc.name(idx) for idx in dist.index]

In [ ]:
dist

In [ ]:
genodist = distdf["109"].sort_values(ascending=False)

In [ ]:
genodist.index = [vc.name(idx) for idx in genodist.index]

In [ ]:
genodist